In [1]:
# 导入包
import pandas as pd

### 加载数据、观察数据

In [2]:
# 加载全量样本
# user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
# user_info = pd.read_csv('./data_format1/user_info_format1.csv')
# train_data1 = pd.read_csv('./data_format1/train_format1.csv')
# submission = pd.read_csv('./data_format1/test_format1.csv')
# train_data = pd.read_csv('./data_format2/train_format2.csv')

'''
train_format1：数据集中只包含 label为0和1的数据。
train_format2：数据集中同时包含 label为0、1和-1的数据。
user_info_format1：用户信息数据集。
user_log_format1：用户日志数据集。
'''

# 文件路径
filePath = sys.path[0]

# 导入数据
df_train1 = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'train_format1.csv', encoding='ISO-8859-1')
df_train2 = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'train_format2.csv', encoding='ISO-8859-1')
df_test1 = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'test_format1.csv', encoding='ISO-8859-1')
df_test2 = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'test_format2.csv', encoding='ISO-8859-1')
df_user_info = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'user_info_format1.csv', encoding='ISO-8859-1')
df_user_log = pd.read_csv(filePath + os.sep + 'datasets' + os.sep + 'user_log_format1.csv', encoding='ISO-8859-1')

In [3]:
# 查看数据规模
# print('----------数据集规模-------------')
print('df_train1 shape: ', df_train1.shape, df_train1.columns)
print('df_train2 shape: ', df_train2.shape, df_train2.columns)
print('df_test1 shape: ', df_test1.shape, df_test1.columns)
print('df_test2 shape: ', df_test2.shape, df_test2.columns)
print('df_user_info shape: ', df_user_info.shape, df_user_info.columns)
print('df_user_log shape: ', df_user_log.shape, df_user_log.columns)

# 查看数据大体情况
# print('-----------数据集字段-------------')
# print('df_train1 head: \n', df_train1.head())
# print('df_train2 head: \n', df_train2.head())
# print('df_test1 head: \n', df_test1.head())
# print('df_test2 head: \n', df_test2.head())
# print('df_user_info head: \n', df_user_info.head())
# print('df_user_log head: \n', df_user_log.head())

# 查看是否有缺失值
# print(df_train1.info(verbose=True,null_counts=True))
# print(df_train2.info(verbose=True,null_counts=True))
# print(df_test1.info(verbose=True,null_counts=True))
# print(df_test2.info(verbose=True,null_counts=True))
# print(df_user_info.info(verbose=True,null_counts=True))
# print(df_user_log.info(verbose=True,null_counts=True))

# 特殊值查看
# print('df_train1 label: \n', df_train1['label'].value_counts())
# print('df_train2 label: \n', df_train2['label'].value_counts())

df_train1 shape:  (260864, 3) Index(['user_id', 'merchant_id', 'label'], dtype='object')
df_train2 shape:  (7030723, 6) Index(['user_id', 'age_range', 'gender', 'merchant_id', 'label',
       'activity_log'],
      dtype='object')
df_test1 shape:  (261477, 3) Index(['user_id', 'merchant_id', 'prob'], dtype='object')
df_test2 shape:  (7027943, 6) Index(['user_id', 'age_range', 'gender', 'merchant_id', 'label',
       'activity_log'],
      dtype='object')
df_user_info shape:  (424170, 3) Index(['user_id', 'age_range', 'gender'], dtype='object')
df_user_log shape:  (54925330, 7) Index(['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id', 'time_stamp',
       'action_type'],
      dtype='object')


### 数据预处理

In [4]:
df_train1['origin'] = 'train'
df_test1['origin'] = 'test'
matrix = pd.concat([df_train1, df_test1], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(df_user_info, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6.0,0.0
1,34176,121,0.0,train,6.0,0.0
2,34176,4356,1.0,train,6.0,0.0
3,34176,2217,0.0,train,6.0,0.0
4,230784,4818,0.0,train,0.0,0.0
...,...,...,...,...,...,...
522336,228479,3111,NaN,test,6.0,0.0
522337,97919,2341,NaN,test,8.0,1.0
522338,97919,3971,NaN,test,8.0,1.0
522339,32639,3536,NaN,test,0.0,0.0


In [5]:
# 格式化
df_user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)  # 使用merchant_id（原列名seller_id）
df_user_log['user_id'] = df_user_log['user_id'].astype('int32')
df_user_log['merchant_id'] = df_user_log['merchant_id'].astype('int32')
df_user_log['item_id'] = df_user_log['item_id'].astype('int32')
df_user_log['cat_id'] = df_user_log['cat_id'].astype('int32')
df_user_log['brand_id'].fillna(0, inplace=True)
df_user_log['brand_id'] = df_user_log['brand_id'].astype('int32')
df_user_log['time_stamp'] = pd.to_datetime(df_user_log['time_stamp'], format='%H%M')

matrix['age_range'].fillna(0, inplace=True)  # 0 and NULL for unknown
matrix['gender'].fillna(2, inplace=True)  # 2 and NULL:unknown
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6,0
1,34176,121,0.0,train,6,0
2,34176,4356,1.0,train,6,0
3,34176,2217,0.0,train,6,0
4,230784,4818,0.0,train,0,0
...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0
522337,97919,2341,nan,test,8,1
522338,97919,3971,nan,test,8,1
522339,32639,3536,nan,test,0,0


### 特征处理

In [6]:
# User特征处理
groups = df_user_log.groupby(['user_id'])

# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')

# 统计操作类型为0，1，2，3的个数，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10
0,34176,3906,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
1,34176,121,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
2,34176,4356,1.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
3,34176,2217,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
4,230784,4818,0.0,train,0,0,54,31,17,20,19,5.166667,47.0,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,282,6.000000,1770.0,NaN,26.0,208.0
522337,97919,2341,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522338,97919,3971,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522339,32639,3536,nan,test,0,0,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0


In [7]:
# 商家特征处理
groups = df_user_log.groupby(['merchant_id'])

# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

# 统计商家被交互的action_type 唯一值，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

# 按照merchant_id 统计随机负采样的个数
temp = df_train2[df_train2['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10
0,34176,3906,0.0,train,6,0,451,256,45,109,...,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,2861
1,34176,121,0.0,train,6,0,451,256,45,109,...,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,4530
2,34176,4356,1.0,train,6,0,451,256,45,109,...,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,1088
3,34176,2217,0.0,train,6,0,451,256,45,109,...,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,7268
4,230784,4818,0.0,train,0,0,54,31,17,20,...,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,3102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,...,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,1982
522337,97919,2341,nan,test,8,1,55,29,14,17,...,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,703
522338,97919,3971,nan,test,8,1,55,29,14,17,...,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,3050
522339,32639,3536,nan,test,0,0,72,46,24,33,...,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,2177


In [8]:
# 按照user_id, merchant_id分组
groups = df_user_log.groupby(['user_id', 'merchant_id'])

# 统计行为个数
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

# 统计item_id, cat_id, brand_id唯一个数
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

# 统计不同action_type唯一个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

# 统计时间间隔
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,...,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9
0,34176,3906,0.0,train,6,0,451,256,45,109,...,2861,39,20,6,1,36.0,NaN,1.0,2.0,0.850000
1,34176,121,0.0,train,6,0,451,256,45,109,...,4530,14,1,1,1,13.0,NaN,1.0,NaN,0.050000
2,34176,4356,1.0,train,6,0,451,256,45,109,...,1088,18,2,1,1,12.0,NaN,6.0,NaN,0.016667
3,34176,2217,0.0,train,6,0,451,256,45,109,...,7268,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
4,230784,4818,0.0,train,0,0,54,31,17,20,...,3102,8,1,1,1,7.0,NaN,1.0,NaN,0.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,...,1982,5,2,1,1,4.0,NaN,1.0,NaN,0.016667
522337,97919,2341,nan,test,8,1,55,29,14,17,...,703,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
522338,97919,3971,nan,test,8,1,55,29,14,17,...,3050,16,5,2,1,12.0,NaN,4.0,NaN,0.150000
522339,32639,3536,nan,test,0,0,72,46,24,33,...,2177,3,2,1,1,2.0,NaN,1.0,NaN,0.000000


In [9]:
# 用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 

# 商家购买点击比 不需要这个特征值，得分0.68
matrix['r2'] = matrix['m8']/matrix['m6'] 

# 不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']

# 填充缺失值
matrix.fillna(0, inplace=True)

# 采用one-hot编码 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)

# 采用one-hot编码 修改gender字段名称为 g_0, g_1, g_2
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)

# 删除age_range, gender字段
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,34176,3906,0.0,train,451,256,45,109,108,5.833333,...,0,0,0,0,1,0,0,1,0,0
1,34176,121,0.0,train,451,256,45,109,108,5.833333,...,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1.0,train,451,256,45,109,108,5.833333,...,0,0,0,0,1,0,0,1,0,0
3,34176,2217,0.0,train,451,256,45,109,108,5.833333,...,0,0,0,0,1,0,0,1,0,0
4,230784,4818,0.0,train,54,31,17,20,19,5.166667,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,6.000000,...,0,0,0,0,1,0,0,1,0,0
522337,97919,2341,nan,test,55,29,14,17,17,4.750000,...,0,0,0,0,0,0,1,0,1,0
522338,97919,3971,nan,test,55,29,14,17,17,4.750000,...,0,0,0,0,0,0,1,0,1,0
522339,32639,3536,nan,test,72,46,24,33,35,5.800000,...,0,0,0,0,0,0,0,1,0,0


### 训练集和测试集划分

In [10]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

### 模型训练（传统模型）

In [17]:
# 导入用到的模型包
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

In [12]:
# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42     
)
model.fit(
    X_train, y_train,
    eval_metric='auc', 
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)
model.fit(X_train, y_train)

[0]	validation_0-auc:0.63820	validation_1-auc:0.63155
[1]	validation_0-auc:0.65231	validation_1-auc:0.64245
[2]	validation_0-auc:0.65894	validation_1-auc:0.64660
[3]	validation_0-auc:0.66400	validation_1-auc:0.64916
[4]	validation_0-auc:0.66476	validation_1-auc:0.65022
[5]	validation_0-auc:0.66624	validation_1-auc:0.65222
[6]	validation_0-auc:0.66811	validation_1-auc:0.65462
[7]	validation_0-auc:0.66784	validation_1-auc:0.65405
[8]	validation_0-auc:0.66892	validation_1-auc:0.65513
[9]	validation_0-auc:0.67090	validation_1-auc:0.65639
[10]	validation_0-auc:0.67171	validation_1-auc:0.65604
[11]	validation_0-auc:0.67403	validation_1-auc:0.65787
[12]	validation_0-auc:0.67554	validation_1-auc:0.65954
[13]	validation_0-auc:0.67736	validation_1-auc:0.66036
[14]	validation_0-auc:0.67837	validation_1-auc:0.66082
[15]	validation_0-auc:0.68172	validation_1-auc:0.66295
[16]	validation_0-auc:0.68452	validation_1-auc:0.66521
[17]	validation_0-auc:0.68622	validation_1-auc:0.66656
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=300, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [18]:
# 使用LightGBM模型
model = lgb.LGBMClassifier(
    num_leaves=51,
    max_depth=10,
    boosting_type='gbdt',
    objective='binary',
    learning_rate=0.015,
    n_estimators=2000,
    subsample=0.75,
    subsample_freq=2,
    reg_lambda=0.28,
    reg_alpha=0.12,
    colsample_bytree=0.8,
    min_child_samples=300,
    min_split_gain=0.1
)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_metric='auc', 
    early_stopping_rounds=100
)

578	training's binary_logloss: 0.198099	valid_1's auc: 0.684798	valid_1's binary_logloss: 0.215524
[866]	training's auc: 0.798683	training's binary_logloss: 0.198078	valid_1's auc: 0.684832	valid_1's binary_logloss: 0.215521
[867]	training's auc: 0.798794	training's binary_logloss: 0.198058	valid_1's auc: 0.684863	valid_1's binary_logloss: 0.215518
[868]	training's auc: 0.79889	training's binary_logloss: 0.19804	valid_1's auc: 0.684853	valid_1's binary_logloss: 0.215517
[869]	training's auc: 0.79897	training's binary_logloss: 0.198023	valid_1's auc: 0.684848	valid_1's binary_logloss: 0.215517
[870]	training's auc: 0.799064	training's binary_logloss: 0.198006	valid_1's auc: 0.684854	valid_1's binary_logloss: 0.215518
[871]	training's auc: 0.799199	training's binary_logloss: 0.197984	valid_1's auc: 0.684884	valid_1's binary_logloss: 0.215514
[872]	training's auc: 0.799295	training's binary_logloss: 0.197965	valid_1's auc: 0.684889	valid_1's binary_logloss: 0.215514
[873]	training's auc: 

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.015, max_depth=10,
               min_child_samples=300, min_split_gain=0.1, n_estimators=2000,
               num_leaves=51, objective='binary', reg_alpha=0.12,
               reg_lambda=0.28, subsample=0.75, subsample_freq=2)

### 测试集预测

In [20]:
prob = model.predict_proba(test_data)
df_test1['prob'] = pd.Series(prob[:,1])
# df_test1.drop(['origin'], axis=1, inplace=True)
df_test1.to_csv('prediction2.csv', index=False)

- xgbboost 目前测试集最高得分0.6774169，prediction1.csv
- lightgbm 目前测试集最高得分0.6827759，prediction2.csv